# 🔄 Pairs Trading with Meridian v2.1.2

**Cross-Market Arbitrage Engine - Stage 1**

This notebook demonstrates how to use the intermarket arbitrage module for pairs trading based on cycle synchronization.

## Features:
- Select tradable pairs based on cycle correlation
- Detect divergence opportunities
- Generate trading signals
- Backtest strategies with realistic execution

**Author:** Meridian Team  
**Date:** December 4, 2025  
**Status:** Stage 1 Complete


## 1. Setup and Imports


In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import yfinance as yf

# Add src to path
sys.path.insert(0, '../src')

from meridian_v2_1_2.intermarket_arbitrage import (
    PairsSelector,
    DivergenceDetector,
    PairsStrategy,
    PairsBacktester
)

print("✅ Imports successful")


## 2. Fetch Market Data

We'll fetch data for common intermarket pairs:
- **GLD/SLV**: Gold vs Silver
- **TLT/IEF**: Long-term vs Medium-term Treasuries  
- **SPY/QQQ**: S&P 500 vs Nasdaq


In [ ]:
# Define asset universe
symbols = ['GLD', 'SLV', 'TLT', 'IEF', 'SPY', 'QQQ']

# Date range (2 years of data)
end_date = datetime.now()
start_date = end_date - timedelta(days=365*2)

# Fetch data
price_dict = {}
for symbol in symbols:
    try:
        ticker = yf.Ticker(symbol)
        hist = ticker.history(start=start_date, end=end_date)
        if len(hist) > 0:
            price_dict[symbol] = hist['Close']
            print(f"✅ {symbol}: {len(hist)} days")
    except Exception as e:
        print(f"❌ {symbol}: {e}")

print(f"\n📊 Fetched {len(price_dict)} instruments")


## 3. Screen for Tradable Pairs

Use `PairsSelector` to find pairs with high cycle correlation and mean-reverting spreads.


In [ ]:
# Initialize selector
selector = PairsSelector(
    min_correlation=0.6,
    max_lead_lag_days=10
)

# Find top pairs
pairs = selector.select_pairs(price_dict, top_n=5)

# Display results
print(f"\n🎯 Found {len(pairs)} tradable pairs:\n")
for i, pair in enumerate(pairs, 1):
    print(f"{i}. {pair.lead_asset}/{pair.lag_asset}")
    print(f"   Correlation: {pair.correlation:.3f}")
    print(f"   Score: {pair.score:.3f}\n")


## 4. Generate Signals and Backtest

Generate trading signals and run a backtest with the best pair.


In [ ]:
if pairs:
    selected_pair = pairs[0]
    print(f"📊 Analyzing: {selected_pair.lead_asset}/{selected_pair.lag_asset}\n")
    
    # Initialize strategy and backtester
    strategy = PairsStrategy(entry_threshold=2.0, exit_threshold=0.5)
    backtester = PairsBacktester(initial_capital=100000)
    
    # Get prices
    lead_prices = price_dict[selected_pair.lead_asset]
    lag_prices = price_dict[selected_pair.lag_asset]
    
    # Run backtest
    result = backtester.backtest(
        pair_candidate=selected_pair,
        strategy=strategy,
        lead_prices=lead_prices,
        lag_prices=lag_prices
    )
    
    print("\n🎯 Backtest Results:\n")
    for key, value in result.summary().items():
        print(f"{key}: {value}")
else:
    print("No pairs found")


## 5. Visualize Results


In [ ]:
if pairs and result.total_trades > 0:
    plot_data = backtester.plot_results(result)
    
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(plot_data['dates'], plot_data['equity'], 'b-', linewidth=2, label='Equity')
    ax.axhline(y=plot_data['initial_capital'], color='gray', linestyle='--', 
               label='Initial Capital')
    ax.set_title('Equity Curve')
    ax.set_xlabel('Date')
    ax.set_ylabel('Equity ($)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


## ✅ Stage 1 Complete!

The Cross-Market Arbitrage Engine is operational. Try experimenting with different:
- Asset pairs
- Entry/exit thresholds
- Risk parameters

**Next Steps:** Implement Stage 2 (Cycle Regime Classifier) to improve signal quality!
